# Visualisation of `RWC_alam`

*Arthur Ryman*
*Last Updated: 2022-05-13*

## Introduction

I am testing `acmpy`.
The function RWC_alam minimises a certain function.
The test case for `RWC_alam(B, -3, 2)` passes for `B in range(0, 15)` but fails for`B >= 15`.
The Maple version passes for all values of `B`.

The Python version uses the SymPy function `solveset`.
Is this a bug or limitation?

What is going on here?

Try plotting the function that is being optimised.

## Mathematical Specification

The mathematical theory for ACM is described in \[WR2015\].

The function `RWC_alam` minimizes the expectation value of the RWC Hamiltonian defined by the real parameters $B$, $c_1$, $c_2$, and $\kappa$. 
We are interested in the case:

$$
\begin{gather}
B > 0 \\
c_1 < 0 \\
c_2 > 0 \\
\kappa = 0
\end{gather}
$$

The test cases all use the following values.

In [1]:
c1: float = -3.0
c2: float = 2.0
kappa: float = 0.0

The test cases pass for $B \le 14$ and fails for $B \ge 15$.
We'll examine the first failing value.

In [2]:
B: float = 15.0

### $\beta_0(c_1, c_2)$

The parameter $\beta_0$ is defined in (B.15).

$$
\beta_0(c_1, c_2) = 
\left\{ 
\begin{array}{ll}
\sqrt{-\frac{c_1}{2 c_2}} & \mbox{if $c_1 < 0$} \\
0 & \mbox{otherwise}
\end{array}
\right.
$$

In [3]:
import math

def beta0(c1: float, c2: float) -> float:
    """(B.15)"""
    assert c2 > 0.0
    return 0.0 if c1 >= 0 else math.sqrt(-c1 / (2 * c2))

beta_0 = beta0(c1, c2)
beta_0

0.8660254037844386

### $\lambda_0(a, \beta_0)$

The parameter $\lambda_0$ is defined in (B.11).

$$
\lambda_0 = 1 + \sqrt{9/4 + (a \beta_0)^4}
$$

In [4]:
def lambda0(a: float, beta0: float) -> float:
    """(B.11)"""
    assert beta0 > 0.0
    return 1 + math.sqrt(9 / 4 + (a * beta0) ** 4)

[lambda0(a, beta_0) for a in range(1, 21)]

[2.6770509831248424,
 4.354101966249685,
 7.914658342969666,
 13.093386622447824,
 19.809904305976673,
 28.041634565979923,
 37.78059950571768,
 49.02343178074636,
 61.768515696863936,
 76.01499850029991,
 91.76239584761962,
 109.01041616436815,
 127.75887542890239,
 148.0076528620194,
 169.7566665349846,
 193.00585928559573,
 217.75519024927638,
 244.00462958552865,
 271.7541550927704,
 301.00374997656274]

For small $a$ we have:

$$
\begin{split}
\lambda_0
& = 1 + \sqrt{9/4 + (a \beta_0)^4} \\
& \approx 1 + \sqrt{9/4} \\
& = 1 + 3/2 \\
& = 5/2
\end{split}
$$

For large $a$ we have:

$$
\begin{split}
\lambda_0
& = 1 + \sqrt{9/4 + (a \beta_0)^4} \\
&\approx (a \beta_0)^2
\end{split}
$$

### $E(a, \lambda_0, B, c_1, c_2, \kappa)$

Let $E$ denote the expectation value of the RWC Hamiltonian on the ground state. 
The value E is defined in (B.16).

$$
\begin{align}
E_1 &= \frac{a^2}{2B}\left[ 1 + \frac{9}{4(\lambda_0 - 1)} \right] \\
E_2 &= \frac{B}{2 a^2}c_1 \lambda_0 \\
E_3 &= \frac{B}{2 a^4}c_2 \lambda_0(\lambda_0 + 1) \\
E_4 &= \frac{\kappa}{3} \\
E &= E_1 + E_2 + E_3 + E_4
\end{align}
$$

In [6]:
def RWC_ham_expt(a: float, lambda0: float, B: float, c1: float, c2: float, kappa: float
                ) -> float:
    """(B.16)"""
    E1: float = (a ** 2 / (2 * B)) * (1 + 9 / (4 * (lambda0 - 1)))
    E2: float = B * c1 * lambda0 / (2 * a ** 2)
    E3: float = B * c2 * lambda0 * (lambda0 + 1) / (2 * a ** 4)
    E4: float = kappa / 3
    return E1 + E2 + E3 + E4

[RWC_ham_expt(a, lambda0(a, beta_0), B, c1, c2, kappa) for a in range(1, 21)]

[87.49920181319229,
 -2.4137607276439113,
 -6.32301325716336,
 -6.967699762085397,
 -7.002085904161742,
 -6.8005401089500115,
 -6.461583879389282,
 -6.020850550401022,
 -5.493941329031037,
 -4.888618829984798,
 -4.20911782034675,
 -3.4579152013997714,
 -2.636541212484234,
 -1.7459840012954029,
 -0.7869054188135145,
 0.24023738899701463,
 1.335120266828877,
 2.4975080843103203,
 3.7272268711572476,
 5.024145639356021]

For small $a$, $\lambda_0 \approx 5/2$ so the contribution from $E_3$ dominates.

$$
\begin{split}
E
& \approx E_3 \\
& = \frac{B}{2 a^4}c_2 \lambda_0(\lambda_0 + 1) \\
& \approx \frac{B}{2 a^4}c_2 (5/2)(7/2) \\
& = \frac{35 B}{8 a^4}c_2
\end{split}
$$

For large $a$, $\lambda_0 \approx (a \beta_0)^2$ so the contribution from $E_1$ dominates.

$$
\begin{split}
E
& \approx E_1 \\
& = \frac{a^2}{2B}\left[ 1 + \frac{9}{4(\lambda_0 - 1)} \right] \\
& \approx \frac{a^2}{2B}\left[ 1 + \frac{9}{4((a \beta_0)^2 - 1)} \right] \\
& \approx \frac{a^2}{2B}
\end{split}
$$

It is therefore apparent that $B$ must have a minimum for some finite, positive value
of $a$.
Plot $E$ as a function of $a$ for several values of $B$.